In [2]:
strength = 0.5
ddim_steps = 50


t_enc = int(strength * ddim_steps)
print(f"target t_enc is {t_enc} steps")

target t_enc is 25 steps


In [6]:
from einops import rearrange, repeat
import torch

In [7]:
repeat(torch.randn((1, 3, 4)), '1 ... -> b ...', b=10).shape

torch.Size([10, 3, 4])

In [ ]:
@torch.no_grad()
def encode_first_stage(self, x):
    return self.first_stage_model.encode(x)

def get_first_stage_encoding(self, encoder_posterior):
    if isinstance(encoder_posterior, DiagonalGaussianDistribution):
        z = encoder_posterior.sample()
    elif isinstance(encoder_posterior, torch.Tensor):
        z = encoder_posterior

    return self.scale_factor * z

In [ ]:
# 여기가 vae

content_image = repeat(content_image, '1 ... -> b ...', b=batch_size)
# 왜 두번 latent => posterior sample
content_latent = model.get_first_stage_encoding(model.encode_first_stage(content_image))
init_latent = content_latent

In [8]:
def extract_into_tensor(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

In [ ]:
class DDIMSampler(object):
    def __init__(self, model, schedule="linear", **kwargs):
        super().__init__()
        self.model = model
        self.ddpm_num_timesteps = model.num_timesteps
        self.schedule = schedule

    @torch.no_grad()
    def stochastic_encode(self, x0, t, use_original_steps=False, noise=None):
        if use_original_steps: # 원본 DDIM alphas를 사용함
            sqrt_alphas_cumprod = self.sqrt_alphas_cumprod
            sqrt_one_minus_alphas_cumprod = self.sqrt_one_minus_alphas_cumprod
        else:
            sqrt_alphas_cumprod = torch.sqrt(self.ddim_alphas)
            sqrt_one_minus_alphas_cumprod = self.ddim_sqrt_one_minus_alphas

        if noise is None:
            noise = torch.randn_like(x0)
        return (extract_into_tensor(sqrt_alphas_cumprod, t, x0.shape) * x0 + 
                extract_into_tensor(sqrt_one_minus_alphas_cumprod, t, x0.shape) * noise)

    def make_schedule(self, ddim_num_steps, ddim_discretize="uniform", ddim_eta=0., verbose=True):
        self.ddim_timesteps = make_ddim()

    

In [ ]:
sampler = DDIMSampler(model)

In [ ]:
sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)

In [ ]:
uc = model.get_learned_conditioning(batch_size * [""], style_image)


c= model.get_learned_conditioning(prompts, style_image)


# stochastic inversion
t_enc = int(strength * 1000)

if scale != 1.0:
    uc = model.get_learned_conditioning(batch_size * [""], style_image)
if isinstance(prompts, tuple):
    prompts = list(prompts)

c= model.get_learned_conditioning(prompts, style_image)


t_enc = int(strength * 1000)
x_noisy = model.q_sample(x_start=init_latent, t=torch.tensor([t_enc]*batch_size).to(device))
model_output = model.apply_model(x_noisy, torch.tensor([t_enc]))
z_enc = sampler.stochastic_encode(init_latent, torch.tensor())